## Train ResNets

### created by Yuying Liu, 04/30/2020

This script is a template for training neural network time-steppers for different systems and different time scales. To reproduce the results in the paper, one needs to obtain all 11 neural network models for each nonlinear system under study. For setup details, please refer to Table 2 in the paper.

In [2]:
import os
import sys
import torch
import numpy as np

import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import ResNet as net

In [3]:
# adjustables

t = np.linspace(0, 8*np.pi, 500)
k = 0                         # model index: should be in {0, 2, ..., 10}
dt = t[1] - t[0]#0.01# 0.0005                     # time unit: 0.0005 for Lorenz and 0.01 for others
system = 'toy1'#_4x4'         # system name: 'Hyperbolic', 'Cubic', 'VanDerPol', 'Hopf' or 'Lorenz'
# total_dim = 16
noise = 0.0                   # noise percentage: 0.00, 0.01 or 0.02

lr = 1e-3                     # learning rate
max_epoch = 100000            # the maximum training epoch 
batch_size = 320              # training batch size
# arch = [total_dim, 128, 128, 128, total_dim]  # architecture of the neural network

In [4]:
# paths
data_dir = os.path.join('../data/', system)
model_dir = os.path.join('../model/', system)

level = 0
filter_group_num = 0
encoded = np.load(os.path.join(data_dir, 'data_L{}_{}.npy'.format(level, filter_group_num)))

print(encoded.shape)
n_steps, _, n_per_dim, _ = encoded.shape
total_dim = n_per_dim **2
arch = [total_dim, 128, 128, 128, total_dim] 

# global const
n_forward = 5
step_size = 2**k

(5000, 1, 3, 3)


In [9]:
# load data
data = np.load(os.path.join(data_dir, "train_data.npy"))#inputs_2x2.npy"))#'train_noise{}.npy'.format(noise)))

print(data.shape)

(100, 500, 4)
train_data = encoded[:2500,0,:,:].reshape((1,2500,9))
val_data = encoded[2500:3750,0,:,:].reshape((1,1250,9))
test_data = encoded[3750:,0,:,:].reshape((1,1250,9))

# print(np.load("../../data/Hyperbolic/train_noise0.0.npy").shape)
# data = np.expand_dims(data, 1)
# train_data = data[:75]
# val_data = data[75:90]
# test_data = data[90:]
# val_data = np.load(os.path.join(data_dir, 'val_noise{}.npy'.format(noise)))
# test_data = np.load(os.path.join(data_dir, 'test_noise{}.npy'.format(noise)))
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)
n_train = train_data.shape[0]
n_val = val_data.shape[0]
n_test = test_data.shape[0]
print(n_train)
print(n_val)
print(n_test)

print(train_data[0,:,:])

print("step_size*n_forward+1 = ", step_size*n_forward+1)
# create dataset object
dataset = net.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)

(100, 500, 4)
(1, 2500, 9)
(1, 1250, 9)
(1, 1250, 9)
1
1
1
[[0.7202411  0.7316998  0.7302419  ... 0.7134938  0.724209   0.72017264]
 [0.72024053 0.7316991  0.73024106 ... 0.71343565 0.72415453 0.720172  ]
 [0.72023875 0.7316972  0.73023903 ... 0.71337503 0.72409767 0.7201702 ]
 ...
 [0.72023726 0.73169565 0.73023766 ... 0.71362925 0.7243348  0.7201687 ]
 [0.7202397  0.73169833 0.73024035 ... 0.71357685 0.7242862  0.72017115]
 [0.72024107 0.73169965 0.7302416  ... 0.7135221  0.72423524 0.7201725 ]]
step_size*n_forward+1 =  6


In [5]:
# for i in range(100):
#     plt.plot(train_data[i,:,0], train_data[i,:,1], 'b')

In [10]:
for k in [ 0, 1, 2, 3, 4, 5]:
    print("k = ", k)
    step_size = 2**k
    dataset = net.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)
    print(dataset.train_x.shape)
    model_name = 'model_D{}_noise{}_0.pt'.format(step_size, noise)

    # create/load model object
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model = torch.load(os.path.join(model_dir, model_name), map_location=device)
        model.device = device
    except:
        print('create model {} ...'.format(model_name))
        model = net.ResNet(arch=arch, dt=dt, step_size=step_size)

    # training
    model.train_net(dataset, max_epoch=30000, batch_size=batch_size, lr=lr,
                    model_path=os.path.join(model_dir, model_name))

k =  0
torch.Size([1, 9])
create model model_D1_noise0.0_0.pt ...
self.n_dim=  9
dataset.n_dim =  9
epoch 1000, training loss 1.2766795433327705e-11, validation loss 2.4129477929624166e-11
(--> new model saved @ epoch 1000)
--> model has reached an accuracy of 1e-8! Finished training!
k =  1
torch.Size([1, 9])
create model model_D2_noise0.0_0.pt ...
self.n_dim=  9
dataset.n_dim =  9
epoch 1000, training loss 1.8597998141522964e-10, validation loss 2.2648004305292346e-10
(--> new model saved @ epoch 1000)
--> model has reached an accuracy of 1e-8! Finished training!
--> new model saved @ epoch 1001
k =  2
torch.Size([1, 9])
create model model_D4_noise0.0_0.pt ...
self.n_dim=  9
dataset.n_dim =  9
epoch 1000, training loss 1.874024935233365e-09, validation loss 1.969896690212636e-09
(--> new model saved @ epoch 1000)
--> model has reached an accuracy of 1e-8! Finished training!
--> new model saved @ epoch 1001
k =  3
torch.Size([1, 9])
create model model_D8_noise0.0_0.pt ...
self.n_dim= 

epoch 11000, training loss 7.071535605973622e-07, validation loss 6.753234060852265e-07
(--> new model saved @ epoch 11000)
epoch 12000, training loss 4.831653086512233e-07, validation loss 4.737930510145816e-07
(--> new model saved @ epoch 12000)
epoch 13000, training loss 3.3403318866476184e-06, validation loss 3.412713567740866e-06
epoch 14000, training loss 5.624761456601846e-07, validation loss 5.518149919225834e-07
epoch 15000, training loss 3.2369626978834276e-07, validation loss 3.155631418394478e-07
(--> new model saved @ epoch 15000)
epoch 16000, training loss 6.047761758054548e-07, validation loss 5.835442493662413e-07
epoch 17000, training loss 4.081916245013417e-07, validation loss 3.9822649000598176e-07
epoch 18000, training loss 2.283374982425812e-07, validation loss 2.2163706603350875e-07
(--> new model saved @ epoch 18000)
epoch 19000, training loss 6.01978285885707e-07, validation loss 6.265152023843257e-07
epoch 20000, training loss 1.7863165169273998e-07, validation

In [7]:
 

.

SyntaxError: invalid syntax (<ipython-input-7-a5d5b61aa8a6>, line 1)